<a href="https://colab.research.google.com/github/lbhargav/Convolutional-Neural-Networks/blob/master/tensorflow_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow Basics


In [0]:
# access the data on drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
import tensorflow as tf

In [0]:
# get the version of tensorflow
print(tf.__version__)

1.12.0


In [0]:
# simple algebra
a = tf.constant(8.0)
b = tf.constant(6.0)
c = a * b
with tf.Session() as sess:
   output = sess.run(c)
   print(output)

48.0


In [0]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = a * b
with tf.Session() as sess:
  output = sess.run(c, {a:[4,5], b: [9, 4]})
  print(output)

[36. 20.]


In [0]:
# linear model
# varibles and placeholders
w = tf.Variable(6.0, tf.float32)
b = tf.Variable(5.0, tf.float32)
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
y_hat= w * x + b

#loss function
mean_square = tf.square(y - y_hat)
loss = tf.reduce_sum(mean_square)

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)


init = tf.global_variables_initializer()

with tf.Session() as sess:
  sess.run(init)
  for i in range (100):
    output = sess.run(train, {x:[4,5], y: [9, 4]})
  print(sess.run([w,b]))     

[0.2831888, 4.9457994]


### MNIST Dataset

In [0]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random

In [0]:
dataset = pd.read_csv("drive/My Drive/MNIST Digit Dataset/train.csv", nrows = 1000)
dataset.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
Y = pd.get_dummies(dataset["label"])
Y.head(5)
dataset = dataset.drop("label", axis=1)
print(dataset.head(5))

   pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   pixel9    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0         0   

In [0]:
# splitting data into train and validation set
X_train, X_valid, Y_train, Y_valid = train_test_split(dataset, Y,test_size = 0.2)

In [0]:
# initilizing weights
W1 = tf.Variable(tf.random_normal([784, 300], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([300]), name='b1')
# and the weights connecting the hidden layer to the output layer
W2 = tf.Variable(tf.random_normal([300, 10], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([10]), name='b2')

# Python optimisation variables
learning_rate = 0.5
epochs = 10
batch_size = 100

# declare the training data placeholders
# input x - for 28 x 28 pixels = 784
x = tf.placeholder(tf.float32, [None, 784])
# now declare the output data placeholder - 10 digits
y = tf.placeholder(tf.float32, [None, 10])


# calculate the output of the hidden layer
Z = tf.add(tf.matmul(x, W1), b1)
A = tf.nn.relu(Z)

y_ = tf.nn.softmax(tf.add(tf.matmul(A, W2), b2))

y_clipped = tf.clip_by_value(y_, 1e-10, 0.9999999)
cross_entropy = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped)
                         + (1 - y) * tf.log(1 - y_clipped), axis=1))


# add an optimiser
optimiser = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cross_entropy)


# finally setup the initialisation operator
init_op = tf.global_variables_initializer()

# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


# start the session
with tf.Session() as sess:
   # initialise the variables
    sess.run(init_op)
    total_batch = int(len(Y) / batch_size)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = X_train, Y_train
            _, c = sess.run([optimiser, cross_entropy], 
                         feed_dict={x: batch_x, y: batch_y})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
    print(sess.run(accuracy, feed_dict={x: X_valid, y: Y_valid}))


Epoch: 1 cost = 34.261
Epoch: 2 cost = 34.925
Epoch: 3 cost = 34.925
Epoch: 4 cost = 34.925
Epoch: 5 cost = 34.925
Epoch: 6 cost = 34.925
Epoch: 7 cost = 34.925
Epoch: 8 cost = 34.925
Epoch: 9 cost = 34.925
Epoch: 10 cost = 34.925
0.07
